In [1]:
DATA_IN = "../data/raw/rees46/_/"
DATA_OUT = "../data/raw/rees46/"

import findspark
findspark.init()

In [2]:
import pyspark 
from pyspark.sql import SparkSession

# later on - probably add dtypes in load phase
from pyspark.sql.types import StringType, IntegerType,\
    DoubleType, StructType, StructField, DateType

In [3]:
sc = pyspark.SparkContext()
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '10g'), ('spark.executor.cores', '4'),
    ('spark.cores.max', '4'), ('spark.driver.memory','8g'), ("spark.kryoserializer.buffer.max","1g"),
    ("spark.sql.execution.arrow.pyspark.enabled", "true")])
sc.stop()
sc = pyspark.SparkContext(conf=conf)

ss = SparkSession.builder.getOrCreate()
# terminate # set parames # get or create
ss.sparkContext.setLogLevel("ERROR")
ss.sparkContext.getConf().getAll()

22/03/17 10:42:44 WARN Utils: Your hostname, mf-H110M-S2H resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface enx00d2b176ce73)
22/03/17 10:42:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/17 10:42:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[('spark.executor.memory', '10g'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '45283'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.executor.cores', '4'),
 ('spark.cores.max', '4'),
 ('spark.app.id', 'local-1647510166789'),
 ('spark.kryoserializer.buffer.max', '1g'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.driver.host', '192.168.0.108'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.execution.arrow.pyspark.enabled', 'true'),
 ('spark.app.startTime', '1647510166722')]

In [4]:
def get_sdf_info(sdf, name=""):
    rc = sdf.count(); cc = len(sdf.columns) # note - wrap this into a fuc
    print("The {} dataset has shape of ({},{}), and following dtypes\n{}".\
        format(name, rc, cc, sdf.dtypes))

In [ ]:
import glob
for f in glob.glob(DATA_IN+"*.gz"):
    if "events" not in locals():
        events = ss.read.csv(f,header=True, inferSchema=True, nanValue="null")
    else:
        events.union(ss.read.csv(f,header=True, inferSchema=True, nanValue="null"))
#events = ss.read.csv(glob.glob(DATA_DIR+"*.gz")[2],header=True, inferSchema=True)
get_sdf_info(events)
events.show(3)

In [5]:
events = ss.read.csv(DATA_OUT+"events", header=True)

In [6]:
# keep interactions for users with >=10 trans
from pyspark.sql.functions import col, countDistinct
users_to_keep = events.filter(col("event_type")=="purchase").\
    groupBy("user_id").agg(countDistinct("user_session").\
        alias("n_transactions")).filter(col("n_transactions")>=10).select(col("user_id"))
events = events.join(users_to_keep, ["user_id"], "inner")
get_sdf_info(events)

The  dataset has shape of (1806685,6), and following dtypes
[('user_id', 'string'), ('event_time', 'string'), ('event_type', 'string'), ('product_id', 'string'), ('price', 'string'), ('user_session', 'string')]


In [ ]:
# encode cats
def encode_column(df, column_name, replace=False):
    from pyspark.sql.functions import col, lit, create_map
    from itertools import chain
    vals = [c[column_name]for c in df.select(column_name).distinct().collect()]
    vals_dict = {v:i for i,v in enumerate(vals)}
    map_exp =  create_map([lit(x) for x in chain(*vals_dict.items())])
    fcol = "enc_"
    if replace:
        fcol = ""
    return df.withColumn(fcol+column_name,
        map_exp.getItem(col(column_name)))

In [7]:
# carry-out date conversion and remapping
from pyspark.sql.functions import col,to_timestamp
events = events.withColumn("event_time", to_timestamp(col("event_time")))
#cols_to_replace = ["user_session", "user_id", "product_id", "category_id"]
#for c in cols_to_replace:
#    events = encode_column(events, c, replace=True)
get_sdf_info(events)
events.show(3)

The  dataset has shape of (1806685,6), and following dtypes
[('user_id', 'string'), ('event_time', 'timestamp'), ('event_type', 'string'), ('product_id', 'string'), ('price', 'string'), ('user_session', 'string')]


+---------+-------------------+----------+----------+------+--------------------+
|  user_id|         event_time|event_type|product_id| price|        user_session|
+---------+-------------------+----------+----------+------+--------------------+
|512389344|2019-12-02 10:39:34|      view|   1005177|849.16|8f3c8447-ed83-4c0...|
|512389344|2019-12-02 10:39:36|      cart|   1005177|849.16|8f3c8447-ed83-4c0...|
|512389344|2019-12-02 10:41:46|  purchase|   1005177|849.16|8f3c8447-ed83-4c0...|
+---------+-------------------+----------+----------+------+--------------------+
only showing top 3 rows



In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
cols_to_replace = ["user_id", "product_id", "category_id", "event_type"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index")
    for column in cols_to_replace]
pipeline = Pipeline(stages=indexers)
pipeline = pipeline.fit(events)
events = pipeline.transform(events)

Py4JJavaError: An error occurred while calling o150.fit.
: org.apache.spark.SparkException: Input column category_id does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:245)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:245)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:242)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spa

In [ ]:
# carve-out some of the info as it is duplicated
products = events.select(["product_id", "category_id", "brand"]).dropDuplicates()
categories = events.select(["category_id", "category_code"]).dropDuplicates()
events = events.select(["event_time", "event_type", "user_id", "product_id", "price", "user_session"])
get_sdf_info(products, "products")
get_sdf_info(categories, "categories")
get_sdf_info(events, "events")

In [ ]:
# try to save both of the datasets to gz
#import os, shutil
#shutil.rmtree(DATA_OUT, ignore_errors=True)
#os.makedirs(DATA_OUT)
#events.write.csv(DATA_OUT+"events", compression="gzip", header=True)
#products.write.csv(DATA_OUT+"products", compression="gzip", header=True)
#categories.write.csv(DATA_OUT+"categories", compression="gzip", header=True)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol="category_id", outputCol="category_id_encoded")
si = si.fit(categories)
si.transform(categories).show()

In [ ]:
products = ss.read.csv(DATA_OUT+"products", header=True)
products.show()

In [ ]:
si.transform(products).filter(col("category_id")==2053013552259662037).show()

In [ ]:
from pyspark.sql.functions import col

In [ ]:
# the product table - both of them small enough to do this firstly in pandas and rewriting it to spark

# product id
# category id
# collapsed brand name

# category table

# category id
# category name

In [ ]:
# test on tree-based mapping
import pandas as pd
cat_df = ss.read.csv(DATA_OUT+"categories", header=True).toPandas()

# we might use this mapping
def get_edges(leaf_id, category_code):
    if category_code is None:
        cat=[]
    else:
        cat = category_code.split(".")
    res = []
    prev = leaf_id
    while len(cat)>0:
        curr = cat.pop()
        res.append([curr, prev])
        prev=curr
    if len(cat)==0:
        res.append([None,prev])
    return pd.DataFrame(res,
        columns=["parent_category", "category"])

tree = []
for i,r in cat_df.iterrows():
    tree.append(get_edges(r["category_id"], r["category_code"]))
pd.concat(tree)        